<a href="https://colab.research.google.com/github/Ahtesham519/jupyter-notebook_python/blob/main/Text_cllassification_using_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy tensorflow torch scikit-learn pytorch_pretrained_bert pytorch-nlp tqdm pandas matplotlib beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 18.7 MB/s eta 0:00:00


In [2]:
#importing a few nessary pacakages and setting the DATA directory 
DATA_DIR="."
import warnings
warnings.filterwarnings("ignore")
import os 
import numpy as np
import pickle
import tensorflow as tf

In [4]:

#Bert imports 
import torch
from torch.utils.data import TensorDataset, DataLoader , RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam , BertForSequenceClassification

from tqdm import tqdm, trange
import pandas as pd
import io 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline



In [5]:
#specify GPU device
'''device = torch.device("cuba" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available:
  n_gpu = torch.cuda.device_count()
  torch.cuda.get_device_name(0)
  '''
  

RuntimeError: ignored

In [10]:
#uploading and reading the datasets
#Source for the dataset is kaggle 

try:
  from google.colab import files
  #after downloading the datsets upload the dataset here
 
  df = pd.read_csv("IMDB Dataset.csv" , engine = 'python' , error_bad_lines=False)

except ModuleNotFoundError:

  #After downloading the dataset , put the IMDB dataset file in the data folder.
  df = pd.read_csv("Data/IMDB Dataset.csv" ,engine = 'python' , error_bad_lines =False)

In [11]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [12]:
from sklearn.preprocessing import LabelEncoder

#sentimental is positive and negative we need to convert it to 0,1
le = LabelEncoder()
df["sentiment"] = le.fit_transform(df["sentiment"])

In [13]:
df['sentiment'].value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

In [14]:
!pip install bs4
from bs4 import BeautifulSoup

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1270 sha256=229aa7cd924ce0c25a7285c4b3e46ab2b82260d3417d2d232e5170bd5e89ca9d
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [15]:
#cleaning the text

import re

def strip(text):
  soup = BeautifulSoup(text , "html.parser")
  text = re.sub('\[[^]]*\]' ,' ', soup.get_text())
  pattern = r"[^a-zA-z0-9\s,']"
  text= re.sub(pattern, '' ,text)
  return text

df['review'] = df['review'].apply(strip)

In [16]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production The filming tech...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,Petter Mattei's Love in the Time of Money is a...,1


In [17]:
#BERT needs to understand 2 things:
#1 The start and the end of each sentiment
#so we declare a special token CLS which tells BERT that its a classification task 

sentences =  df['review']
sentence =["[CLS]  "+ i + " [SEP]" for i in sentences]

In [18]:
sentence[0]

"[CLS]  One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked They are right, as this is exactly what happened with meThe first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO Trust me, this is not a show for the faint hearted or timid This show pulls no punches with regards to drugs, sex or violence Its is hardcore, in the classic use of the wordIt is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda Em City is home to manyAryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and moreso scuffles, death stares, dodgy dealings and shady agreements are never far awayI would say the main appeal of the show is due to the fact that it goes where other shows wouldn't d

In [20]:
!pip install sentencepiece transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.8 MB/s eta 0:00:00


In [19]:
#Tokennize with Bert tokenizer
tokenizer = BertTokenizer.from_pretrained('bert.base-uncased' , do_lower_case = True)

#Restricting the max size of tokens to 512(Bert doesent accept any more than this)

tokenized_texts = list(map(lambda t: tokenizer.tokenize(t)[:510] , sentence ))

print("Tokenize the first sentence: ")
print( tokenized_texts[0])

ERROR:pytorch_pretrained_bert.tokenization:Model name 'bert.base-uncased' was not found in model name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese). We assumed 'bert.base-uncased' was a path or url but couldn't find any file associated to this path or url.


AttributeError: ignored

In [ ]:
labels = list(df['sentiment'])   #Storing the labels


In [ ]:
#set the maximum sequence length
MAX_LEN = 128

#Pad our input tokens so that everything has a uniform length 
im=nput_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids , tokenized_texts)),
                            maxlen= MAX_LEN , dtype = "long" , truncating= "post" , padding = "post")

In [8]:
#Use the Bert tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids , maxlen=MAX_LEN , dtype = "long" , truncating = "post" , padding = "post")

NameError: ignored

bert is MLM

In [9]:
#Create attention masks 
attention_masks = [] 

#Create a mask of its 1s for each token followed by  0s for padding

for seq in input_ids:
  seq_mask = [float (i>0) for i in seq]
  attention_masks.append(seq_mask)

NameError: ignored

Now we will split the data into train and validatation. Convert it to the tensor and then create iterator for our data.



In [21]:
#Select a batch size for training.

batch_size = 16

#Use train test_split to split our data into train and validation sets for training
train_inputs, validation_inputs, train_labels , validation_labels = train_test_split(input_ids, labels, 
                 random_state = 2018 , test_size = 0.1)

train_masks , validation_masks , _ , _  = train_test_split(attention_masks , input_ids, 
                                                           random_state = 2018 , test_size=0.1)

#convert all our data into torch tensors, the required datatype for our model .
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks - torch.tensor(train_masks)
validation_masks - torch.tensor(validation_masks)


#Create an iterator of our data with the torch DataLoader
train_data = TensorDataset(train_inputs , train_masks , train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data , sampler= train_sampler , batch_size = batch_size)
validation_data = TensorDataset(validation_inputs , validation_masks , validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data , sampler = validation_sampler , batch_size = batch_size)


SyntaxError: ignored

In [22]:
#Loading the pretrained BERT

model = BertForSequenceClassification.from_pretrained("bert-base-uncased" , num_labels = 2 )  #Binary classification
if torch.cuda.is_available():
  print(model.cuda())
else:
  print(model)

100%|██████████| 407873900/407873900 [00:05<00:00, 77142720.39B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=Fa

In [23]:
#Bert fine tuning parameters
param_optimizer = list(model.named_parameters())
no_decay = ['bias' , 'gamma' , 'beta']
optimizer_grouped_parameters = [
    {'params' : [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate' : 0.01 },
    {'params' : [p for n ,p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate' : 0.0}

] 

optimizer = BertAdam(optimizer_grouped_parameters , 
                     lr = 2e-5,
                     warmup = .1)

#Function to calculate the accuracy of our preedictions vs labels 
def flat_accuracy(preds , labels):
  pred_flat = np.argmax(preds , axis=1) .flatten()
  labels_flat = labels.flatten()
  return np.sum(pred_flat == labels_flat) / len(labels_flat)


torch.cuda.empty_cache()
#Store our loss and accuracy for plotting

train_loss_set = []

#Number of training epochs
epochs = 4

#Bert training loop 
for _ in trange(epochs , desc = "Epoch"):
  ##Training 


  #Set our model to training mode 
  model.train()
  #Tracking variables 
  tr_loss = 0
  nb_tr_examples , nb_tr_steps = 0 , 0

  #Train the data for one epoch
  for step , batch in enumerate(train_dataloader):
    #Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    #Unpack the inputs from our dataloader
    b_input_ids , b_input_mask , b_labels = batch
    #Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    #forward pass
    loss = model(b_input_ids , token_type_ids = None , attention_masks = b_input_mask , labels = b_labels)
    train_loss_set.append(loss.item())
    #Backward pass
    loss.backward()
    #update parameters and take a step using the computed gradient
    optimizer.step()
    #update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1
  print("Train loss: {}" .format(tr_loss/nb_tr_steps))


  #VALIDATION

  #Put model in evaluation mode
  model.eval()
  #Tracking variables
  eval_loss , eval_accuracy = 0,0
  nb_eval_steps , nb_eval_examples = 0,0

  #Evulate data for one epoch
  for batch in validation_dataloader:
    #Add batch to GPU 
    batch = tuple(t.to(device) for t in b_input_ids)
    #unpack the inputs from our dataloader
    b_input_ids, b_input_mask , b_labels = batch
    #Telling the model not to compute or store gradients , saving memory and speeding up validation 
    with torch.no_grad():
      #forward pass, calculate logit predictions
      logits - model(b_input_ids  , token_type_ids = None , attention_masks = b_input_masks)
    #Move logits and labels to GPU
    logits = logits.detach().cpu().numpy()
    labels_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy = flat_accuracy(logits , labels_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1
  print("validation Accuracy:{} " .format(eval_accuracy/nb_eval_steps))


  #Plot training performance
plt.figure(figsize = (15, 8))
plt.title("Training Loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(train_loss_set)
plt.show()
    

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]


NameError: ignored